# Prevendo o preço do aluguel no Rio de Janeiro
- A empresa HashMoveis contratou para ajudar a decidir em quais imóveis poderá investir
  - Prever qual seria o aluguel desse imóvel pelo airbnb e investir nos imóveis que irão trazer um rápido retorno de investimento

Principais etapas do projeto:
1. Definição do problema
2. Entendimento e tratamento de dados
3. Análise Exploratória
4. Tratamento dos Dados
5. Definição do modelo
6. Modelo em produção

# Definição do Problema
- Para entender o problemam algumas perguntas são fundamentais:
 - De forma direta: "Qual o problema que eu quero resolver?"
 - Quais são as minhas métricas de sucesso?
 - Qual o prazo desejado para esse projeto?
 - Quem são os interessados nesse resultado?
 - Quais são os **dados** disponíveis

# Entendimento e tratamento dos dados

In [126]:
# Importando a base de dados
import pandas as pd

In [127]:
# Importando uma base de dados utilizando o pandas
dados = pd.read_csv("quinto_andar_transformed_data.csv")

**Importante analisar as informações dos dados .info()**

In [ ]:
# Visualizando as informações da base de dados
#type(dados)
dados.info()

In [ ]:
# Verificando valores nulos
dados.isnull().sum()

In [ ]:
#Filtrando esses valores
dados[dados.condominio.isnull()]

In [ ]:
# Podemos, por exemplo, acessar esses anúncios
dados.url[2287]

In [ ]:
#Selecionando apenas esse coluna
dados.loc[dados.condominio.isnull(), 'condominio']

In [133]:
# Substituindo esses valores por 0
dados.loc[dados.condominio.isnull(), 'condominio'] = 0

In [ ]:
# Verificando novamente os valores nulos
dados.isnull().sum()

In [ ]:
# Fazendo o mesmo processo para iptu
#Filtrando esses valores nulos
dados[dados.iptu.isnull()]

In [ ]:
dados.url[580]

In [ ]:
# Verificando quantas linhas temos nulas na base
dados[dados.iptu.isnull()]

In [138]:
# Podemos excluir essas linhas que estão sem valor de iptu
dados = dados.dropna(axis=0)

In [ ]:
# Verificando quantas linhas ficamos na base
dados.shape

In [ ]:
# Para garantir que não temos valores vazios
dados.isnull().sum()

**O que é a primeira coluna?**

Eliminando as colunas desnecessárias

In [ ]:
# Selecionando a coluna especifica da base
dados.url

In [ ]:
# Eliminando essa coluna
dados = dados.drop('url', axis=1)
display(dados)

## Análise Exploratória de Dados
- Nessa etapa, vamos explorar nossa base

In [ ]:
# Relembrando a visualização da base
dados.head(3)

**Muito importante ver o resumo estatístico da base de dados (.describe())**

In [ ]:
# Entendendo os dados estatísticos
dados.describe()

In [ ]:
# Somando todos os valores de aluguel
dados.aluguel.sum()

In [ ]:
# Contando a quantidade de valores
dados.aluguel.count()

In [ ]:
dados.aluguel.sum()/dados.aluguel.count()

In [ ]:
# Calculando a media (mean), desvio padrão(std), mínimo (min) e máximo (máx)
dados.aluguel.max()

In [ ]:
dados.aluguel.median()

In [ ]:
# Contando o número de registros antes e depois de 1950
dados[dados.aluguel >= 1950]

In [ ]:
# Verificando 75%
dados.aluguel.quantile(0.75)

In [ ]:
# Plotando um histograma
dados.aluguel.hist(bins = 15)

In [ ]:
# Melhorando a plotagem do gráfico
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,5))
ax.hist(dados.aluguel,bins=60, linewidth =0.5, edgecolor="white")
ax.set(xlim=(400, 11500), xticks=[500,1400,1950,2746.5,11250])
plt.show()

In [ ]:
# Podemos verificar quais os intervalos e números de valores em cada faixa
import numpy as np
np.histogram(dados.aluguel, bins=60)

In [ ]:
# Uma outra forma de apresentar os dados
dados.aluguel.plot.box(vert=False)

In [ ]:
# Verificando as 10 propriedades de maior valor
dados.sort_values('aluguel', ascending=False).head(10)

In [ ]:
# Verificar se tem valores duplicados na base
dados[dados.duplicated()]

In [158]:
# Elimando valores duplicados
dados = dados.drop_duplicates(keep='last')

In [ ]:
# Verificando novamente a base
# Verificando as 10 propriedades de maior valor
dados.sort_values('aluguel', ascending=False).head(10)

In [ ]:
#Será que existe relação entre aluguel e metragem?
dados.plot.scatter(x='metragem', y='aluguel');

In [ ]:
#Será que existe relação entre aluguel e quartos?
dados.plot.scatter(x='quarto', y='aluguel');

## Uso do seaborn

In [162]:
# Importando seaborn
import seaborn as sns

In [ ]:
# Qual será a média do aluguel entre propriedades próximas e distantes do metrô?
sns.boxplot(x='metro_prox',y='aluguel',data=dados)

In [ ]:
# Podemos avaliar o bairro
fig, ax = plt.subplots(figsize=(15,15))
sns.boxplot(x='aluguel',y='bairro',data=dados, orient='h')

In [ ]:
# Verificando a quantidade de registros em cada bairro
dados.bairro.value_counts()

In [ ]:
# Importando uma base de bairros x regiões
regioes = pd.read_excel('bairros.xlsx')
display(regioes)

In [167]:
#Unindo essas duas base de dados
base2 = pd.merge(
 dados,
 regioes,
 how='left',
 left_on='bairro',right_on='Bairro'
)

In [ ]:
base2.head(3)

In [ ]:
# Filtrando os valores onde a Região é nula
base2[base2.Região.isnull()]

In [ ]:
# Verificando se existe Vila Maria na base que importamos
regioes[regioes.Bairro == 'Vila Maria']

In [ ]:
# Verificando na base 2 apenas o bairro
base2.loc[base2.Região.isnull(),'bairro']

In [172]:
# Adicionando a região de Vila Maria a Norte
base2.loc[base2.Região.isnull(),'Região'] = 'Norte'

In [173]:
# Corrigindo o nome de Vila Maria
base2.loc[base2.bairro == 'Vila Maria ','bairro'] = 'Vila Maria'

In [ ]:
# Verificando novamente se existe algum valor nulo
base2[base2.Região.isnull()]

In [ ]:
# Agora podemos traçar o gráfico por região
fig, ax = plt.subplots(figsize=(15,15))
sns.boxplot(x='aluguel',y='Região',data=base2, orient='h')

In [ ]:
#Será que existe relação entre aluguel e metragem para a região oeste?
base2[base2.Região == 'Oeste'].plot.scatter(x='metragem', y='aluguel');

In [ ]:
#Será que existe relação entre aluguel e metragem para a região leste?
base2[base2.Região == 'Leste'].plot.scatter(x='metragem', y='aluguel');

In [ ]:
# Criando um gráfico geral de todas regiões
fig, ax = plt.subplots()

oeste = base2[base2.Região == 'Oeste']
leste = base2[base2.Região == 'Leste']
norte = base2[base2.Região == 'Norte']
centro = base2[base2.Região == 'Centro']
sul = base2[base2.Região == 'Sul']

ax.scatter(oeste.aluguel, oeste.metragem, c='b',alpha=0.5, label='oeste')
ax.scatter(leste.aluguel, leste.metragem, c='r',alpha=0.5, label='leste')
ax.scatter(norte.aluguel, norte.metragem, c='g',alpha=0.5, label='norte')
ax.scatter(centro.aluguel, centro.metragem, c='y',alpha=0.5, label='centro')
ax.scatter(sul.aluguel, sul.metragem, c='m',alpha=0.5, label='sul')

ax.legend()

plt.show()

In [ ]:
# Criando um gráfico para cada região
fig, ax = plt.subplots(ncols=3,nrows=2,figsize=(12,7))

oeste = base2[base2.Região == 'Oeste']
leste = base2[base2.Região == 'Leste']
norte = base2[base2.Região == 'Norte']
centro = base2[base2.Região == 'Centro']
sul = base2[base2.Região == 'Sul']

ax[0,0].scatter(oeste.aluguel, oeste.metragem, c='b',alpha=0.5, label='oeste')
ax[0,1].scatter(leste.aluguel, leste.metragem, c='r',alpha=0.5, label='leste')
ax[0,2].scatter(norte.aluguel, norte.metragem, c='g',alpha=0.5, label='norte')
ax[1,0].scatter(centro.aluguel, centro.metragem, c='y',alpha=0.5, label='centro')
ax[1,1].scatter(sul.aluguel, sul.metragem, c='m',alpha=0.5, label='sul')

ax[0,0].set(xlim=(0, 12000),ylim=(0, 600))
ax[0,1].set(xlim=(0, 12000),ylim=(0, 600))
ax[0,2].set(xlim=(0, 12000),ylim=(0, 600))
ax[1,0].set(xlim=(0, 12000),ylim=(0, 600))
ax[1,1].set(xlim=(0, 12000),ylim=(0, 600))

ax[0,0].set_title('oeste')
ax[0,1].set_title('leste')
ax[0,2].set_title('norte')
ax[1,0].set_title('centro')
ax[1,1].set_title('sul')

plt.show()

**Existe alguma relação entre as variáveis**

In [ ]:
base2 = base2.drop(['bairro','Bairro','Região'],axis=1)
base2.head()

In [ ]:
# Verificando se há correlação da base de dados
base2.corr()

In [ ]:
# Tornando a correlação mais visual usando o seaborn
fig, ax = plt.subplots(figsize=(12,7))
sns.heatmap(base2.corr(), annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Podemos verificar graficamente a relação entre algumas variáveis
base3 = dados[['aluguel', 'condominio', 'iptu', 'taxa_serviço','total']]
sns.pairplot(base3)

In [ ]:
# Verificandp o outlier do total
dados.total.max()

In [ ]:
# Filtrando essa linha da base de dados
dados[dados.total == dados.total.max()]

In [186]:
# Importando a base de dados novamente para verificar a informação
verificar = pd.read_csv('quinto_andar_transformed_data.csv')

In [ ]:
# Encontrando o valor de maior iptu
verificar[verificar.iptu == verificar.iptu.max()]

In [ ]:
# Tratando o valor errado
verificar.url[1707]

In [189]:
# Trocando o valor errado pelo correto
dados.loc[dados.iptu == dados.iptu.max(), 'iptu'] = 495.8

In [190]:
dados.loc[dados.total == dados.total.max(), 'total'] = 4227.8

In [ ]:
dados.head()

In [ ]:
#Fazendo novamente
base2 = dados.drop('bairro', axis=1)
# Tornando a correlação mais visual usando o seaborn
fig, ax = plt.subplots(figsize=(12,7))
sns.heatmap(base2.corr(), annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
# Fazendo novamente
# Podemos verificar graficamente a relação entre algumas variáveis
base3 = dados[['aluguel', 'condominio', 'iptu', 'taxa_serviço','total']]
sns.pairplot(base3);

# TRATAMENTO DE DADOS

In [ ]:
# Visualizando novamente a base de dados
dados = base2
dados.head(3)

In [198]:
# Verificando a taxa de serviço
dados['taxa_ser'] = round((dados.taxa_serviço/dados.aluguel)*100,2)

In [ ]:
# Contando os valores
dados.taxa_ser.value_counts()

In [200]:
# Verificando o mesmo para seguro incêndio
dados['taxa_inc'] = round((dados.seguro_incendio/dados.aluguel)*100,2)

In [ ]:
# Contando os valores
dados.taxa_inc.value_counts()

A taxa de serviço e o seguro incêndio são percentuais do aluguel, então para ter a taxa é necessário primeiro ter o valor do aluguel.
Da mesma forma o total também é a soma dos valores já apresentados então colocar o total seria repetir informação.

In [202]:
# Podemos então apagar essa coluna e a coluna que calculamos o percentual
dados = dados.drop(['taxa_serviço', 'taxa_ser', 'seguro_incendio','taxa_inc','total'], axis=1)

In [203]:
# Fazendo a mesma lógica para o Iptu
dados['taxa_iptu'] = round((dados.iptu/dados.aluguel),2)

In [ ]:
# Contando os valores
dados.taxa_iptu.value_counts()

In [205]:
# Mantemos a coluna de iptu, mas retiramos a taxa que usamos para comparar
dados = dados.drop('taxa_iptu', axis=1)

In [ ]:
# Buscando e verificando a base novamente
dados.head(3)

In [ ]:
# Verificando as informações da base
dados.info()

In [ ]:
# Apagando a coluna bairro
dados = dados.drop(['bairro','Bairro'], axis=1)

In [ ]:
# Utilizando o get_dummies
pd.get_dummies(dados.Região)

In [234]:
# Utilizando o concat
dados = pd.concat([dados,pd.get_dummies(dados.Região)], axis=1)

In [ ]:
dados.head()

In [236]:
# Podemos então apagar a única variável que está como texto
dados = dados.drop('Região', axis=1)

**Voltamos nesse processo sempre que necessário durante a criação do modelo!**

#  Definição de modelo

**Aprendizado Supervisionado**: temos exemplos com resultados(rótulos/labels), do que queremos prever.

- **Regressão**: Prever um valor numérico contínuo
 - Exemplo: Prever o valor de uma casa, do aluguel, a quantidade de vendas...
 
- **Classificação**: Prever uma classe
 - Exemplo: Prever se a fruta é banana ou maçã, se uma transação é fraude, se a pessoa possuí ou não uma doença...
 
- Se não temos os valores dos resultados e queremos descobrir apenas estruturas padrões semelhantes, temos um problema de **aprendizado não supervisionado**.
 - Exemplo: Agrupar clientes, definir cesta de produtos, agrupar frutas sem saber qual é a maça ou banana...
 

**Antes de definir o modelo, o primeiro passo é separar os nossos dados em treino e teste**

Esta etapa é extremamente importante  pois vamos precisar **avaliar a qualidade do nosso modelo**

**Vamos utilizar o train_test_split do próprio Scikit_Learn**

In [240]:
# Primeiramente vamos separar o que são nossas variáveis preditoras(x, que vamos usar para fazer) e a variável alvo(que queremos prever, o y)
base = dados
x = base.drop('aluguel', axis=1)
y = base.aluguel

In [239]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [241]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
x_train.head(3)

### Depois de separar a base de treino e teste, podemos usar a base de treino para criar nosso modelo usando diferentes algoritmos

**Usando Regressão linear**

In [2]:
# Importando a regressão linear
from sklearn.linear_model import LinearRegression

In [246]:
# Treinando o nosso modelo
reg = LinearRegression().fit(x_train, y_train)

In [ ]:
# Avaliando o ajuste do modelo aos dados de Treino
reg.score(x_train, y_train)

In [5]:
# Importando o erro médio absoluto
from sklearn.metrics import mean_absolute_error

In [249]:
# Fazendo a previsão com o modelo anterior
y_pred = reg.predict(x_test)

In [ ]:
# Avaliando o erro para a regressão linear
mean_absolute_error(y_test, y_pred)

In [ ]:
# Visualizando a diferença graficamente

fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(y_test, y_pred)
ax.plot([0,10000],[0,10000])

plt.show()

In [257]:
# Importando a árvore de decisão
from sklearn import tree

In [258]:
# Criando o regressor
regtree = tree.DecisionTreeRegressor()

In [259]:
# Fazendo o fit do modelo
regtree = regtree.fit(x_train, y_train)

In [ ]:
# Avaliando o ajuste do modelo aos dados de Treino
regtree.score(x_train, y_train)

In [261]:
# Fazendo a previsão
y_predtree = regtree.predict(x_test)

In [ ]:
# Avaliando o erro para árvore de decisão
mean_absolute_error(y_test, y_predtree)

In [ ]:
# Visualizando a diferença graficamente

fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(y_test, y_predtree)
ax.scatter(y_test, y_pred)
ax.plot([0,10000],[0,10000])

plt.show()

In [6]:
# Importando o erro quadrático médio
from sklearn.metrics import mean_squared_error
# Importando o erro quadrático médio
from sklearn.metrics import root_mean_squared_error

In [ ]:
# Avaliando para regressão
mean_squared_error(y_test, y_pred)

In [ ]:
# Avaliando para árvore
mean_squared_error(y_test, y_predtree)

**Tentando criar uma nova árvore de decisão**

In [309]:
# Criando uma nova árvore de decisão
regtree2 = tree.DecisionTreeRegressor(random_state=20,criterion='absolute_error', max_depth=5)

In [310]:
# Fazendo o fit do modelo
regtree2 = regtree2.fit(x_train, y_train)

In [311]:
# Fazendo a previsão
y_predtree2 = regtree2.predict(x_test)

In [ ]:
# Avaliando o erro para árvore de decisão
mean_absolute_error(y_test, y_predtree2)

In [ ]:
# Visualizando a diferença graficamente

fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(y_test, y_pred, alpha=0.5)
ax.scatter(y_test, y_predtree, alpha=0.5)
ax.scatter(y_test, y_predtree2, alpha=0.5)

ax.plot([0,10000],[0,10000])

plt.show()

In [314]:
# Utilizando o SUPPORT VECTOR REGRESSION
from sklearn.svm import SVR

In [ ]:
# Fazendo a previsão
regsvr = SVR()
regsvr = regsvr.fit(x_train, y_train)
y_predsvr = regsvr.predict(x_test)
mean_absolute_error(y_test, y_predsvr)

In [317]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Treinando o modelo
regr = RandomForestRegressor(max_depth=50, random_state=0)
regr.fit(x_train, y_train)
y_predRF = regr.predict(x_test)

In [ ]:
#Avaliando as métricas do modelo
mse = mean_absolute_error(y_test, y_predRF)
rmse = root_mean_squared_error(y_test, y_predRF)
print('MSE: ', mse)
print('RMSE: ', rmse)

In [ ]:
# Visualizando a diferença graficamente

fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(y_test, y_pred, alpha=0.5, label='Reg')
ax.scatter(y_test, y_predtree, alpha=0.5, label='Arvore1')
ax.scatter(y_test, y_predtree2, alpha=0.5, label='Arvore2')
ax.scatter(y_test, y_predRF, alpha=0.5, label='Random Forest')

ax.plot([0,10000],[0,10000])

ax.legend()

plt.show()

# Modelo em produção

Persistir o modelo

In [347]:
# Importando o dump do joblib
from joblib import dump

In [ ]:
# Fazendo o dump do modelo que queremos usar
dump(regr, 'regressor.joblib')

In [ ]:
x_test.columns